In [1]:
# Imports
import csv
import glob2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import squidpy as sq
import warnings
from anndata import AnnData
from scipy import sparse

In [2]:
# Globals
PRJ_DIR = "/scratch/gpfs/KANG/sereno/spatialstem"
SRC_DIR = f"{PRJ_DIR}/sourcefiles"
RAW_DIR = f"{SRC_DIR}/raw"
HAD_DIR = f"{SRC_DIR}/h5ad"
INT_DIR = f"{PRJ_DIR}/intermediates"
FIG_DIR = f"{PRJ_DIR}/figs"

In [4]:
# Get paths of H5 datasets.
h5ad_paths = glob2.glob(f"{HAD_DIR}/*.h5ad")
h5ad_paths.sort()

['/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p01_fetalliver.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p01_fetalspleen.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p01_fetalthymus.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p02_largeintestine1.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p02_largeintestine2.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p02_smallintestine1.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p02_smallintestine2.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p03_hsc1.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p03_hsc2.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p03_hsc3.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p03_hsc4.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p03_hsc5.h5ad',
 '/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/h5ad/p03

In [43]:
# TODO: Write gene handling functions:
## Human symbol
## Mouse symbol
## Canine symbol
## 
## 
# Iterates through H5ads, finds format of gene ids, and handles each one based on its format.
# Goal: save output 
for h5ad_path in h5ad_paths:
    h5ad_path_strip = h5ad_path.replace(f"{HAD_DIR}/", "")
    ad = sc.read_h5ad(h5ad_path)
    if ("gene_ids" in ad.var.keys()):
        test_gene = ad.var["gene_ids"].iloc[0]
    elif ("SYMBOL" in ad.var.keys()):
        test_gene = ad.var["SYMBOL"].iloc[0]
    else:
        print(h5ad_path, "Gene ID key not found in vars.")
    gene_id = test_gene[0:4]
    # Detects gene symbols and splits into human or mouse symbols
    if (gene_id[0:3]!="ENS"):
        # Check for lowercase characters anywhere in the test gene
        if any([c for c in test_gene if c.islower()]):
            gene_id = "MMUS"
        else:
            gene_id = "HSAS"
    match gene_id:
        case "ENSG":
            print(test_gene, "Human Gene Code")
            # Handle human gene
        case "ENSM":
            print(test_gene, "Mouse Gene Code")
            # Handle mouse gene
        case "ENSC":
            print(test_gene, "Canine Gene Code")
            # Handle canine gene
        case "MMUS":
            print(test_gene, "Mouse symbol")
            # Handle mouse symbol
        case "HSAS":
            print(test_gene, "Human symbol")
            # Handle human symbol
        case _:
            print(test_gene, "Unidentified")
            # Handle unhandled genes

TSPAN6 Human symbol
TSPAN6 Human symbol
TSPAN6 Human symbol
ENSMUSG00000051951 Mouse Gene Code
ENSMUSG00000051951 Mouse Gene Code
ENSMUSG00000051951 Mouse Gene Code
ENSMUSG00000051951 Mouse Gene Code
ENSG00000243485 Human Gene Code
ENSG00000243485 Human Gene Code
ENSG00000243485 Human Gene Code
ENSG00000243485 Human Gene Code
ENSG00000243485 Human Gene Code
ENSG00000243485 Human Gene Code
ENSG00000243485 Human Gene Code
ENSCAFG00000010945 Canine Gene Code
ENSCAFG00000010945 Canine Gene Code
ENSCAFG00000010945 Canine Gene Code
ENSCAFG00000010945 Canine Gene Code
ENSCAFG00000010945 Canine Gene Code
ENSCAFG00000010945 Canine Gene Code
ENSCAFG00000010945 Canine Gene Code
ENSCAFG00000010945 Canine Gene Code
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
Sox17 Mouse symbol
A1BG Human symbol
A1BG Human symbol
A1BG Human symbol
A1BG Human symbol
A1BG Human symbol
A1BG Hum